# 02 - WV2 Image Classification (Pin9.tif)

This notebook loads the multispectral image `data/satellite/Pin9.tif`, reads reference reflectance signatures from `notebooks/surfaces_data/histogram_reflectances.csv`, and performs a simple minimum-distance classification in the WV2 feature space for selected classes. The output is a colorized map highlighting recognized objects with a legend.


In [ ]:
# Setup paths and imports
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Project root
current_dir = Path.cwd()
project_root = current_dir.parent if current_dir.name == 'notebooks' else current_dir

import sys
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.classification import classify_min_distance



In [ ]:
# Inputs
image_path = str(project_root / 'data' / 'satellite' / 'Pin9.tif')
signatures_csv = str(project_root / 'notebooks' / 'surfaces_data' / 'histogram_reflectances.csv')

# Choose which classes to recognize (subset of CSV 'Object' column)
# Leave as None to use all available
selected_classes = [
    'grass', 'dry_grass', 'weed',
    'concrete', 'black_asphalt',
    'soil', 'rough_soil'
]



In [ ]:
# Run classification
if HAS_RASTERIO:
    class_map, idx_to_class, class_to_color = classify_min_distance(
        image_path=image_path,
        signatures_csv=signatures_csv,
        class_names=selected_classes
    )
    
    # Load one RGB composite for background visualization if possible
    with rasterio.open(image_path) as ds:
        H = ds.height
        W = ds.width
        count = ds.count
        # Try a typical WV2 RGB order: Red, Green, Blue -> indices 5,3,2 in 1-based (approx)
        bands_to_try = [
            (5, 3, 2),   # (Red, Green, Blue)
            (4, 3, 2),   # (Yellow as R fallback)
            (3, 2, 1),   # (Green, Blue, Coastal)
        ]
        rgb = None
        for (r, g, b) in bands_to_try:
            if max(r, g, b) <= count:
                r_arr = ds.read(r).astype(np.float32)
                g_arr = ds.read(g).astype(np.float32)
                b_arr = ds.read(b).astype(np.float32)
                # Simple scaling for display
                def scale_im(x):
                    x = x / (np.nanpercentile(x, 99.5) + 1e-6)
                    return np.clip(x, 0, 1)
                rgb = np.dstack([scale_im(r_arr), scale_im(g_arr), scale_im(b_arr)])
                break
else:
    # Fallback: create dummy classification for demonstration
    print("⚠️ Using dummy classification since rasterio is not available.")
    print("To install rasterio for Python 3.14, try:")
    print("  pip install --find-links https://www.lfd.uci.edu/~gohlke/pythonlibs/ rasterio")
    print("  or use conda: conda install -c conda-forge rasterio")
    
    # Create dummy data
    H, W = 500, 500  # Dummy dimensions
    class_map, idx_to_class, class_to_color = create_dummy_classification(H, W, len(selected_classes))
    rgb = None



In [ ]:
# Visualization helpers
import matplotlib.patches as mpatches

def colorize(class_map, idx_to_class, class_to_color):
    h, w = class_map.shape
    out = np.zeros((h, w, 3), dtype=np.float32)
    for idx, name in idx_to_class.items():
        mask = class_map == idx
        color = class_to_color[name]
        out[mask] = color
    return out

colored = colorize(class_map, idx_to_class, class_to_color)

# Compose plots
fig, axes = plt.subplots(1, 2 if 'rgb' in locals() and rgb is not None else 1, figsize=(14, 6))
if not isinstance(axes, np.ndarray):
    axes = np.array([axes])

if 'rgb' in locals() and rgb is not None:
    axes[0].imshow(rgb)
    axes[0].set_title('RGB composite')
    axes[0].axis('off')
    ax_cls = axes[1]
else:
    ax_cls = axes[0]

ax_cls.imshow(colored)
ax_cls.set_title('Minimum-distance classification')
ax_cls.axis('off')

# Legend
patches = []
for name, color in class_to_color.items():
    patches.append(mpatches.Patch(color=color, label=name))
fig.legend(handles=patches, loc='lower center', ncol=min(4, len(patches)))
plt.tight_layout(rect=(0, 0.05, 1, 1))
plt.show()


In [ ]:
# Save outputs
out_dir = project_root / 'notebooks' / 'surfaces_data'
out_dir.mkdir(exist_ok=True)

if HAS_RASTERIO:
    # Save class map as GeoTIFF aligned to source
    with rasterio.open(image_path) as src:
        profile = src.profile
        profile.update(count=1, dtype='int16')
        out_path = out_dir / 'Pin9_min_distance_classes.tif'
        with rasterio.open(out_path, 'w', **profile) as dst:
            dst.write(class_map.astype('int16'), 1)
else:
    # Save as simple numpy array
    np.save(out_dir / 'Pin9_min_distance_classes.npy', class_map)
    print(f"⚠️ Saved classification as numpy array: {out_dir / 'Pin9_min_distance_classes.npy'}")

# Save color preview
plt.imsave(out_dir / 'Pin9_min_distance_classes.png', colored)
